# **CHAPTER 10**
# **Data Aggregation and Group Operations**

## **10.1 GroupBy Mechanics**

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

In [2]:
df

,key1,key2,data1,data2
0,a,one,-0.784424,-0.903231
1,a,two,0.437252,0.267701
2,b,one,-2.564387,-0.525934
3,b,two,-0.216463,1.450094
4,a,one,1.363218,-0.011384


In [3]:
grouped = df['data1'].groupby(df['key1'])

In [4]:
 grouped

In [5]:
grouped.mean()

key1
a    0.338682
b   -1.390425
Name: data1, dtype: float64

In [6]:
 means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [7]:
means

key1  key2
a     one     0.289397
      two     0.437252
b     one    -2.564387
      two    -0.216463
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,0.289397,0.437252
b,-2.564387,-0.216463


In [9]:
 states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [10]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [11]:
df['data1'].groupby([states, years]).mean()

California  2005    0.437252
            2006   -2.564387
Ohio        2005   -0.500444
            2006    1.363218
Name: data1, dtype: float64

In [12]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.338682,-0.215638
b,-1.390425,0.462080


In [13]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.289397 -0.457307
     two   0.437252  0.267701
b    one  -2.564387 -0.525934
     two  -0.216463  1.450094

In [14]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### **Iterating Over Groups**

In [15]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.784424 -0.903231
1    a  two  0.437252  0.267701
4    a  one  1.363218 -0.011384
b
  key1 key2     data1     data2
2    b  one -2.564387 -0.525934
3    b  two -0.216463  1.450094


In [16]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.784424 -0.903231
4    a  one  1.363218 -0.011384
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.437252  0.267701
('b', 'one')
  key1 key2     data1     data2
2    b  one -2.564387 -0.525934
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.216463  1.450094


In [17]:
pieces = dict(list(df.groupby('key1')))

In [18]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-2.564387,-0.525934
3,b,two,-0.216463,1.450094


In [19]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [20]:
grouped = df.groupby(df.dtypes, axis=1)

In [21]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.784424 -0.903231
1  0.437252  0.267701
2 -2.564387 -0.525934
3 -0.216463  1.450094
4  1.363218 -0.011384
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### **Selecting a Column or Subset of Columns**

In [22]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

In [23]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [24]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.457307
     two   0.267701
b    one  -0.525934
     two   1.450094

In [25]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [26]:
s_grouped

In [27]:
s_grouped.mean()

key1  key2
a     one    -0.457307
      two     0.267701
b     one    -0.525934
      two     1.450094
Name: data2, dtype: float64

### **Grouping with Dicts and Series**

In [28]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [29]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values

In [30]:
people

,a,b,c,d,e
Joe,-0.089610,0.374397,1.138353,-0.214470,1.400239
Steve,0.947311,-1.273002,0.185347,1.592569,-1.103762
Wes,-0.047376,NaN,NaN,1.495172,-0.476474
Jim,-1.828131,-1.923988,0.907800,0.376340,-0.063580
Travis,-0.792678,-1.873546,-0.385185,0.009822,-0.544488


In [31]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [32]:
by_column = people.groupby(mapping, axis=1)

In [33]:
 by_column.sum()

,blue,red
Joe,0.923883,1.685027
Steve,1.777916,-1.429453
Wes,1.495172,-0.523849
Jim,1.284140,-3.815698
Travis,-0.375363,-3.210712


In [34]:
map_series = pd.Series(mapping)

In [35]:
 map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [36]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### **Grouping with Functions**

In [37]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.965117,-1.549590,2.046153,1.657042,0.860186
5,0.947311,-1.273002,0.185347,1.592569,-1.103762
6,-0.792678,-1.873546,-0.385185,0.009822,-0.544488


In [38]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [39]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.089610  0.374397  1.138353 -0.214470 -0.476474
  two -1.828131 -1.923988  0.907800  0.376340 -0.063580
5 one  0.947311 -1.273002  0.185347  1.592569 -1.103762
6 two -0.792678 -1.873546 -0.385185  0.009822 -0.544488

### **Grouping by Index Levels**

In [40]:
 columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                      [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])

In [41]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

In [42]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.811515  0.746220  0.837689  0.359570  0.254230
1      1.088151 -1.454156  0.535731  1.692008 -0.262110
2      0.095602 -0.055406 -0.843298 -0.976943 -1.695406
3      1.525327 -0.514111 -0.027080  0.476099 -0.538840

In [43]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## **10.2 Data Aggregation**

In [44]:
df

,key1,key2,data1,data2
0,a,one,-0.784424,-0.903231
1,a,two,0.437252,0.267701
2,b,one,-2.564387,-0.525934
3,b,two,-0.216463,1.450094
4,a,one,1.363218,-0.011384


In [45]:
grouped = df.groupby('key1')

In [46]:
grouped['data1'].quantile(0.9)

key1
a    1.178025
b   -0.451256
Name: data1, dtype: float64

In [47]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [48]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.147642,1.170932
b,2.347924,1.976028


In [49]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.338682  1.077209 -0.784424 -0.173586  0.437252  0.900235   
b      2.0 -1.390425  1.660233 -2.564387 -1.977406 -1.390425 -0.803444   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.363218   3.0 -0.215638  0.611605 -0.903231 -0.457307 -0.011384   
b    -0.216463   2.0  0.462080  1.397263 -0.525934 -0.031927  0.462080   

                          
           75%       max  
key1                      
a     0.128159  0.267701  
b     0.956087  1.450094

### **Column-Wise and Multiple Function Application**

In [50]:
tips = pd.read_csv('../examples/tips.csv')

In [51]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [52]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [53]:
grouped = tips.groupby(['day', 'smoker'])

In [54]:
grouped_pct = grouped['tip_pct']

In [55]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [56]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [57]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [58]:
functions = ['count', 'mean', 'max']

In [59]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

<ipython-input-59-426117f1e146>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


In [60]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [61]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [62]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [63]:
 grouped['tip_pct', 'total_bill'].agg(ftuples)

<ipython-input-63-c2b2ffd79840>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [64]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [65]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### **Returning Aggregated Data Without Row Indexes**

In [66]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## **10.3 Apply: General split-apply-combine**

In [67]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [68]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [69]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [70]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [71]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [72]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [73]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [74]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### **Suppressing the Group Keys**

In [75]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### **Quantile and Bucket Analysis**

In [76]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

In [77]:
frame

,data1,data2
0,0.199105,-0.024674
1,0.561622,-1.279543
2,-0.493575,-1.297038
3,0.277560,-0.987904
4,-0.848400,-0.463641
...,...,...
995,0.946230,0.507432
996,-0.072745,-0.814502
997,0.666120,-1.362768
998,1.016529,0.150661


In [78]:
quartiles = pd.cut(frame.data1, 4)# 4-> bölme ölçütü

In [79]:
quartiles[:10]

0     (0.132, 1.494]
1     (0.132, 1.494]
2     (-1.23, 0.132]
3     (0.132, 1.494]
4     (-1.23, 0.132]
5     (0.132, 1.494]
6    (-2.597, -1.23]
7     (1.494, 2.856]
8     (-1.23, 0.132]
9     (0.132, 1.494]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.597, -1.23] < (-1.23, 0.132] < (0.132, 1.494] < (1.494, 2.856]]

In [80]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [81]:
grouped = frame.data2.groupby(quartiles)

In [82]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.597, -1.23]",-2.910179,2.334332,102.0,-0.133837
"(-1.23, 0.132]",-2.398197,3.067447,431.0,0.002261
"(0.132, 1.494]",-2.969463,2.511638,394.0,0.051573
"(1.494, 2.856]",-1.954831,1.847990,73.0,-0.044600


In [83]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.910179,2.334332,100.0,-0.123135
1,-2.398197,2.406517,100.0,-0.060260
2,-2.166346,3.067447,100.0,-0.051714
3,-2.242272,2.213172,100.0,-0.084117
4,-2.274619,2.773344,100.0,0.131851
5,-2.183156,2.398278,100.0,0.151245
6,-2.086885,2.455811,100.0,0.137129
7,-2.620014,2.511638,100.0,0.057409
8,-2.969463,2.394328,100.0,-0.053250


### **Example: Filling Missing Values with Group-Specific Values**

In [84]:
s=pd.Series(np.random.randn(6))

In [85]:
s

0    0.325603
1    0.088553
2    1.000349
3   -1.087145
4   -0.225210
5    1.343404
dtype: float64

In [86]:
s[::2]=np.nan

In [87]:
s

0         NaN
1    0.088553
2         NaN
3   -1.087145
4         NaN
5    1.343404
dtype: float64

In [88]:
s.fillna(s.mean())
#NA / NaN değerlerini belirtilen yöntemi kullanarak doldurur.

0    0.114937
1    0.088553
2    0.114937
3   -1.087145
4    0.114937
5    1.343404
dtype: float64

In [89]:
states=['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']

In [90]:
group_key=['East']*4+['West']*4

In [91]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [92]:
data=pd.Series(np.random.randn(8),index=states)

In [93]:
data

Ohio         -0.385370
New York     -0.459489
Vermont      -0.584554
Florida      -0.868932
Oregon        0.722023
Nevada        0.484622
California   -0.486278
Idaho         1.502593
dtype: float64

In [94]:
data[['Vermont','Nevada','Idaho']]=np.nan

In [95]:
data

Ohio         -0.385370
New York     -0.459489
Vermont            NaN
Florida      -0.868932
Oregon        0.722023
Nevada             NaN
California   -0.486278
Idaho              NaN
dtype: float64

In [96]:
data.groupby(group_key).mean()

East   -0.571264
West    0.117873
dtype: float64

Groupby işlemi, nesneyi bölme, bir işlev uygulama ve sonuçları birleştirmenin bazı kombinasyonlarını içerir. Bu, büyük miktarda veriyi gruplamak ve bu gruplar üzerindeki işlemleri hesaplamak için kullanılabilir.

In [97]:
fill_mean=lambda g: g.fillna(g.mean())

In [98]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.385370
New York     -0.459489
Vermont      -0.571264
Florida      -0.868932
Oregon        0.722023
Nevada        0.117873
California   -0.486278
Idaho         0.117873
dtype: float64

In [99]:
fill_values = {'East': 0.5, 'West': -1}

In [100]:
fill_func=lambda g: g.fillna(fill_values[g.name])

In [101]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.385370
New York     -0.459489
Vermont       0.500000
Florida      -0.868932
Oregon        0.722023
Nevada       -1.000000
California   -0.486278
Idaho        -1.000000
dtype: float64

### **Example: Random Sampling and Permutation**

In [102]:
# Hearts, Spades, Clubs, Diamonds
suits=['H','S','C','D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

**Extend () yöntemi**, bir yinelenebilir öğenin tüm öğelerini (liste, tuple, dize vb.) Listenin sonuna ekler.

In [103]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [104]:
def draw(deck, n=5):
    return deck.sample(n)

**sample ()** Python'da, list, tuple, string veya set gibi diziden seçilen öğelerin belirli bir uzunluk listesini döndürendahili bir rastgele modül işlevidir. Değiştirilmeden rastgele örnekleme için kullanılır.

In [105]:
draw(deck)

6H     6
7H     7
2S     2
QC    10
5C     5
dtype: int64

In [106]:
get_suit = lambda card: card[-1] # last letter is suit

In [107]:
deck.groupby(get_suit).apply(draw, n=2)

C  7C      7
   JC     10
D  KD     10
   QD     10
H  7H      7
   10H    10
S  AS      1
   3S      3
dtype: int64

In [108]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

9C     9
6C     6
5D     5
4D     4
5H     5
KH    10
4S     4
2S     2
dtype: int64

### **Example: Group Weighted Average and Correlation**

In [109]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                                               'b', 'b', 'b', 'b'],
                                        'data': np.random.randn(8),
                                        'weights': np.random.rand(8)})

In [110]:
df

,category,data,weights
0,a,-0.195226,0.580012
1,a,-0.992328,0.314788
2,a,0.006363,0.377152
3,a,-1.119237,0.098073
4,b,0.446377,0.550306
5,b,-0.656319,0.484741
6,b,-0.397689,0.536710
7,b,0.284471,0.399797


In [111]:
grouped = df.groupby('category')

In [112]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

**np.average()=** Belirtilen eksen boyunca ağırlıklı ortalamayı hesaplayın.
Ağırlıklı ortalama, her bileşenin önemini yansıtan bir faktörle çarpılmasından elde edilen bir ortalamadır.

In [113]:
grouped.apply(get_wavg)

category
a   -0.389025
b   -0.087349
dtype: float64

In [114]:
 close_px = pd.read_csv('../examples/stock_px_2.csv', parse_dates=True,
                        index_col=0)
 

In [115]:
close_px.info()
#dataframe.info()işlevi, veri çerçevesinin kısa bir özetini almak için kullanılır. 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [116]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [117]:
spx_corr = lambda x: x.corrwith(x['SPX'])

**corrwith=** İkili korelasyon, Series veya DataFrame satırları veya sütunları olan DataFrame'in satırları veya sütunları arasında hesaplanır. DataFrame'ler, korelasyonları hesaplamadan önce her iki eksen boyunca hizalanır.

In [118]:
 rets = close_px.pct_change().dropna()

In [119]:
get_year = lambda x: x.year

In [120]:
by_year = rets.groupby(get_year)

In [121]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [122]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### **Example: Group-Wise Linear Regression**

In [123]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [124]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## **10.4 Pivot Tables and Cross-Tabulation**

In [125]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [126]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [127]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [128]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [129]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### **Cross-Tabulations: Crosstab**

In [137]:
data=pd.DataFrame({'Sample':np.arange(1,11),
                   'Nationality':['USA','Japan','USA','Japan','Japan','Japan','USA','USA','Japan','USA'],
                   'Handedness':['Right-handed','Left-handed','Right-handed','Right-handed','Left-handed','Right-handed',
                                'Right-handed','Left-handed','Right-handed','Right-handed']
                  })

In [138]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [139]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [140]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

## **10.5 Conclusion**